# AI Investment Analyst

An agent that:

* Scrapes financial news + quarterly reports

* Runs sentiment analysis

* Combines with stock fundamentals

* Generates investment theses (bullish/bearish)

* Could integrate with Yahoo Finance API for real-time data.

# Imports

In [1]:
!pip install -U langgraph "langchain[openai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [2]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16
!pip install -U langchain pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 0.6.4 requires langchain-core>=0.3.67, but you have langchain-core 0.3.33 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.2 MB/s eta 0:00:00
  

In [3]:
!pip install langgraph

In [4]:
!pip install yfinance


In [5]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv
import os

# Some Tools That My Model Should Use

In [6]:
f=open("/content/Serper APIkey.txt","r")
S_api_key=f.read()
os.environ["SERPER_API_KEY"] = S_api_key

In [16]:
import yfinance as yf

def ticker_find(tik: str):
  "Returns informations about a stock "
  return yf.Ticker(tik).info




In [11]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

In [17]:
from langchain.agents import Tool

ticket_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search, like if you need to search the ticket name of a stock"
    )
ticker_tool=Tool(name='Ticker',
                 func=ticker_find,
                 description="Useful for when you need information adout the ticket name of a stock")


tools=[]

In [ ]:
ticket_search.invoke('what is the ticket name of Google?')

'GOOG and GOOGL are stock ticker symbols for Alphabet, Google\'s parent company. The main difference between the GOOG and GOOGL symbols is that GOOG shares ... Explore cheap flights from United States to anywhere with Google Flights. Find your next flight, track price changes to get the best deals, and book your ... Add, edit, or remove your ticket & activity details ; To add: Click Get started. Enter your admission or activity details. Ticket or Activity name and Booking ... Google hid its name in the ticket number. 600613 is code for Google. ... Every movie ticket has a serial number, and if Google had its way, every ... To add a pass:\u200b\u200b On your pass issuer app or site, tap Add to Google Wallet. This option can typically be found under "Tickets," "Passes," or "Cards" on the pass ... A one-stop ticketing solution that goes beyond payment. From event tickets, boarding passes to transit tickets, your users can purchase and add all their ... You go to the airline\'s website and

In [18]:
ticker_tool.invoke('GOOG')

{'address1': '1600 Amphitheatre Parkway',
 'city': 'Mountain View',
 'state': 'CA',
 'zip': '94043',
 'country': 'United States',
 'phone': '650-253-0000',
 'website': 'https://abc.xyz',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, devices, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play and YouTube;

# My Model

In [ ]:

f=open("/content/OpenAIKey.txt","r")
api_key=f.read()



llm = ChatOpenAI(
    model_name="moonshotai/kimi-k2:free",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.0)

# Let's Create Our Agent

In [ ]:
#State
class State(TypedDict):
  messages: Annotated[List,add_messages]
